In [12]:
import torch
import torch.nn as nn
import torchvision.models as models
import torch.optim as optim
import os
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader

#device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

from google.colab import drive
drive.mount('/content/drive')

In [13]:
import os

# List the contents of a specific folder in your Drive, for example, a 'datasets' folder
drive_path = "/content/drive/My Drive/Rice_Leaf_Diease/train_new"

if os.path.exists(drive_path):
    files = os.listdir(drive_path)
    print(f"Files in {drive_path}:")
    for file in files:
        print(file)
else:
    print(f"The directory {drive_path} does not exist.")

Files in /content/drive/My Drive/Rice_Leaf_Diease/train_new:
 (1).DS_Store
leaf_blast (1)
narrow_brown_spot (1)
leaf_scald (1)
bacterial_leaf_blight (1)
Tungro (1)
healthy (1)
Sheath Blight (1)
Rice Hispa (1)
brown_spot (1)
Neck_Blast (1)


In [14]:
model=models.resnet34(pretrained=True)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 10)


# Freeze all layers except the final classification layer
#for name, param in model.named_parameters():
#    if 'fc' in name: # Check if the parameter belongs to the final fully connected layer
#        param.requires_grad = True
#    else:
#        param.requires_grad = False

In [15]:
from torchvision import transforms

# Define transformations for training data
train_transforms = transforms.Compose([
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# Define transformations for testing data
test_transforms = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

In [16]:
from torch.utils.data import DataLoader, random_split
from torchvision.datasets import ImageFolder
import torch

# Assume train_transforms is already defined

# 1. Define the path to your data directory
train_dir = '/content/drive/My Drive/Rice_Leaf_Diease/train_new' # Corrected path

# 2. Create the full ImageFolder dataset
full_dataset = ImageFolder(root=train_dir, transform=train_transforms)
print(f"Total images in the dataset: {len(full_dataset)}")

# 3. Define the sizes for your training and validation sets
# Let's use an 80/20 split of the entire dataset
train_size = int(0.8 * len(full_dataset))
val_size = len(full_dataset) - train_size
print(f"Splitting into {train_size} training samples and {val_size} validation samples.")


# 4. Perform the split using random_split
# This ensures the validation set has not been seen during training
# A generator is used for reproducibility of the split
generator = torch.Generator().manual_seed(42) # Use a fixed seed for consistent splits
train_subset, val_subset = random_split(full_dataset, [train_size, val_size], generator=generator)

# 5. Create the DataLoaders from the new Subset objects
# Use clear and distinct variable names
train_loader = DataLoader(
    train_subset,
    batch_size=32,
    shuffle=True,
    num_workers=2, # Reduced number of workers
    pin_memory=True
)

val_loader = DataLoader(
    val_subset,
    batch_size=32,
    shuffle=False,  # No need to shuffle the validation set
    num_workers=2, # Reduced number of workers
    pin_memory=True
)

print("DataLoaders for training and validation have been created successfully.")

Total images in the dataset: 1095
Splitting into 876 training samples and 219 validation samples.
DataLoaders for training and validation have been created successfully.


In [17]:
criterian=nn.CrossEntropyLoss()

In [18]:
import torch.optim as optim
loss_fn=nn.CrossEntropyLoss()
# Define the optimizer
optimizer = optim.Adam(model.parameters(), lr=0.0001)

In [19]:

def train(model,train_loader,loss_fn,optimizer):
    model.train()

    running_loss=0.0
    correc_predict=0
    total_samp=0
    for inputs,labels in train_loader:
        inputs,labels=inputs.to(device),labels.to(device)
        optimizer.zero_grad()
        outputs=model(inputs)
        loss=loss_fn(outputs,labels)
        loss.backward()
        optimizer.step() # Use xm.optimizer_step for TPU # Add mark_step for TPU


        running_loss=loss.item()*inputs.size(0)
        _,predicted=torch.max(outputs.data,1)
        total_samp+=labels.size(0)
        correc_predict+=(predicted==labels).sum().item()
    epoch_loss=running_loss/total_samp
    epoch_acc=100*correc_predict/total_samp

    return epoch_loss,epoch_acc

def evaluate(model,val_loader,loss_fn):
    model.eval()

    running_loss=0.0
    correc_predict=0
    total_samp=0
    with torch.no_grad():
        for inputs,labels in val_loader:
            inputs,labels=inputs.to(device),labels.to(device)

            outputs=model(inputs)
            loss=loss_fn(outputs,labels)

            running_loss=loss.item()*inputs.size(0)
        _,predicted=torch.max(outputs.data,1)
        total_samp+=labels.size(0)
        correc_predict+=(predicted==labels).sum().item()
    epoch_loss=running_loss/total_samp
    epoch_acc=100*correc_predict/total_samp

    return epoch_loss,epoch_acc

**Reasoning**:
I will execute the training loop for a specified number of epochs and evaluate the model periodically. Since the previous steps for setting up the training loop and evaluation are complete, I can combine these into a single block to train and evaluate the model for a few epochs. I will set the number of epochs to 5 and evaluate after each epoch to monitor the training progress.



In [20]:
eproaches=20
model.to(device)
for epoch in range(eproaches):
    train_loss,train_acc=train(model,train_loader,loss_fn,optimizer)
    eval_loss,eval_acc=evaluate(model,val_loader,loss_fn)

    print(f"epoch{epoch+1}:"
          f"train_loss:{train_loss:.2f},train_acc{train_acc:.4f}:"
          f"eval_loss:{eval_loss:.2f},eval_acc{eval_acc:.4f}:")
print("training finished")

epoch1:train_loss:0.01,train_acc75.5708:eval_loss:0.64,eval_acc70.3704:
epoch2:train_loss:0.00,train_acc91.3242:eval_loss:0.31,eval_acc88.8889:
epoch3:train_loss:0.00,train_acc94.6347:eval_loss:0.34,eval_acc92.5926:
epoch4:train_loss:0.01,train_acc93.8356:eval_loss:0.33,eval_acc88.8889:
epoch5:train_loss:0.00,train_acc95.2055:eval_loss:0.53,eval_acc85.1852:
epoch6:train_loss:0.01,train_acc96.3470:eval_loss:0.21,eval_acc92.5926:
epoch7:train_loss:0.00,train_acc96.4612:eval_loss:0.33,eval_acc88.8889:
epoch8:train_loss:0.01,train_acc96.4612:eval_loss:0.28,eval_acc88.8889:
epoch9:train_loss:0.00,train_acc95.7763:eval_loss:0.37,eval_acc92.5926:
epoch10:train_loss:0.00,train_acc97.2603:eval_loss:0.29,eval_acc85.1852:
epoch11:train_loss:0.00,train_acc96.2329:eval_loss:0.31,eval_acc88.8889:
epoch12:train_loss:0.00,train_acc97.6027:eval_loss:0.45,eval_acc85.1852:
epoch13:train_loss:0.00,train_acc97.0320:eval_loss:0.20,eval_acc92.5926:
epoch14:train_loss:0.00,train_acc97.0320:eval_loss:0.26,eval

In [21]:
MODEL_SAVE_PATH = 'rice_leaf_disease.pth'

# Save the model's state dictionary
torch.save(model.state_dict(), MODEL_SAVE_PATH)

print(f"Model saved to {MODEL_SAVE_PATH}")

Model saved to rice_leaf_disease.pth


In [25]:
! python3 predict_custom.py kkl.jpg

Setting up your custom model and environment...
Using device: cpu
Setup complete. Starting prediction...
------------------------------
Image Path:   kkl.jpg
Prediction:   Brown Spot (1)
Confidence:   99.99%
